In [68]:
%%time

%cd "~/lab/NB10422645"
from src.surrogate.sampling import uniform_grid


/home/ggarrett/lab/NB10422645
CPU times: user 1.46 ms, sys: 493 µs, total: 1.95 ms
Wall time: 1.23 ms


In [292]:
%%time

import os


RS_DIR = "~/lab/NB10422645/src/matlab"
RS_RUN = "rs_bicycle"

import subprocess
from scipy.io import savemat, loadmat
import numpy as np

test = np.array([[0, 50, 150, 160],[0, 5, 10, 5]]).astype(float).T
mdict = {'test':test}
savemat('testpy.mat', mdict)

import os 

def pre_constructor(x, v0=0, t0=0, velocity_limit=10, duration=200):
    if len(x)%2==0:
        if len(x)!=2:
            _v = np.concatenate(([v0],x[:len(x)//2])) * velocity_limit
            _t = np.cumsum((np.concatenate(([t0],x[len(x)//2:] + 0.01)))) 
            _t = (_t/np.max(_t)) * duration
        else:
            _v = x * velocity_limit
            _t = np.array([0,1]) * duration
    elif len(x)==3:
        _v = np.concatenate(([v0], x[:2])) * velocity_limit
        _t = np.array([t0, 0.01+ x[-1], 1.01]) * duration
    else:
        raise AssertionError("Nx not recognised.")
    return np.vstack((_t,_v)).T
    
# def setup_designs(sampler, n_samples):
    
def setup_grid_designs(n, N, dir='simulations'):
    le=len(str(int(N**n)))
    os.mkdir(dir) if not os.path.exists(dir) else False
    grid = uniform_grid(np.array([[0]*n,[1]*n]), N, flatten=True)
    for idx, x in enumerate(grid):
#         print(idx)
        mdict = {'x': pre_constructor(x)}
        savemat(f'{dir}/x{idx+1:0{le}}.mat', mdict)
        
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

def run_simulations(dir, simulator='rs_bicycle'):
    files=os.listdir(dir)
    cmds=[f"{simulator} -i {os.path.join(dir, file)} -o {os.path.join(dir,file.replace('x','f'))}" for file in files]
    for batch in chunks(cmds, 100):    
        subprocess_cmd(';'.join(batch))
    
def subprocess_cmd(command):
    process = subprocess.Popen(command,stdout=subprocess.PIPE, shell=True)
    proc_stdout = process.communicate()[0].strip()

def get_results(dir):
    files=os.listdir(dir)
    out=list(filter(lambda x: 'f' in x, files))
    out.sort()
    return np.vstack([loadmat(os.path.join(dir,fout))['rt_f'][-1] for fout in out])
    


setup_grid_designs(3,10, dir='simulations')
# run_simulations('simulations', simulator='src/matlab/rs_bicycle')
f=get_results('simulations')

# f[:,0].reshape(3,3)

CPU times: user 266 ms, sys: 36 ms, total: 302 ms
Wall time: 301 ms


[1.5 4.5]
[  0.          16.66666667 100.         200.        ]


In [319]:
# os.mkdir('simulations2')
# setup_grid_designs(3,20, dir='simulations2')
# run_simulations('simulations2', simulator='src/matlab/rs_bicycle')
f=get_results('simulations2')

# len(f)

8000

In [380]:
### from src.surrogate.sampling import random_uniform
from src.surrogate.sampling.util import plotly3Dsurf, plotly3Dtrisurf
from src.surrogate.test import *
# F2D = TestFunctionSet2DInputSpace()
# F_ = F2D.BukinN6
# F_._bounds = F_.bounds * 1
# N = 100
# f1_grid = uniform_grid(F_.norm_bounds,N, flatten=False)
s =10
# print(f)
f_ = f[:,0].reshape(20,20,20)
# print(f_s
f_ = f_[:,:,s]

grid = uniform_grid(np.array([[0]*3,[1]*3]), 20, flatten=False)
x, y = grid[0][:,:,s], grid[1][:,:,s]
plotly3Dtrisurf(x, y, f_, save=False)


In [195]:
np.meshgrid([1,0,1], [1,0,1])[0].shape

(3, 3)

In [401]:
import plotly.graph_objects as go



fs = f[:,5].reshape(20,20,20).flatten()

trace1 = go.Scatter3d(
    x = grid[0].flatten(),
    y = grid[1].flatten(),
    z = grid[2].flatten(),
    mode='markers',
    marker=dict(
        size=fs/np.max(fs) * 20,
        color=fs,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.3,
        line=dict(width=0),
        showscale=True
    )
)

data = [trace1]
# layout = go.Layout(
#     scene = dict(
#                     xaxis = dict(
#                         title='XXXX-XXXXXX'),
#                     yaxis = dict(
#                         title='XXXX-XXXXXX'),
#                     zaxis = dict(
#                         title='XXXX-XXXXXX'),),
#     margin=dict(
#         r=20, b=10, l=10, t=10
#     )


fig = go.Figure(data=data)
    
fig.show()
#py.iplot(fig, filename='3d-scatter-colorscale')
# plot(fig, filename='D:\\plots\\3dplots\\xx.html')